In [2]:
import os
from google.cloud import translate_v2
import pandas as pd
# from googletrans import Translator

In [4]:
## If you are using colab, you can import google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"/content/drive/MyDrive/JP Morgan/GoogleApiKey.json"

In [6]:
#Data Loading process
train_data = pd.read_csv('/content/drive/MyDrive/JP Morgan/atis_seven_languages/data/train_EN.csv')
test_data = pd.read_csv('/content/drive/MyDrive/JP Morgan/atis_seven_languages/data/test_EN.csv')

In [7]:
#Preprocessing the intent column for both train and test set
def preprocess_intent_column(sentence): 
  sentence = sentence.replace('_', ' ')
  return sentence[5:]

In [8]:
train_data['intent'] = train_data['intent'].apply(preprocess_intent_column)
test_data['intent'] = test_data['intent'].apply(preprocess_intent_column)

In [13]:
train_data[:5]

,id,utterance,slot_labels,intent
0,1,how much does the american airlines flight 71 ...,O O O O B-airline_name I-airline_name O B-flig...,airfare
1,2,could you tell me the cheapest fare from atlan...,O O O O O B-cost_relative O O B-fromloc.city_n...,airfare
2,3,what is the cost of flight dl 106 from philade...,O O O O O O B-airline_code B-flight_number O B...,airfare
3,4,what is the cheapest round trip fare between b...,O O O B-cost_relative B-round_trip I-round_tri...,airfare
4,5,what is the fare from boston to oakland on uni...,O O O O O B-fromloc.city_name O B-toloc.city_n...,airfare


In [14]:
class Translations:
  
  def __init__(self,data, language, target):

    self.data = data
    self.language = language
    self.target = target

  # Performing the translation part

  def process_translations(self,line):

    self.line = line
    
    translate_client = translate_v2.Client()

    translated = translate_client.translate(self.line,target_language= self.target)

    return list(translated.values())[0]

  def translated_data(self):

    for i in self.data.columns[1:4]:
      self.data['{col}_{l}'.format(col = i, l = self.language)]= self.data[i].map(lambda s:self.process_translations(s))     

    return self.data

In [15]:
# Kinyarwanda
trans = Translations(train_data[:5],'Kinyarwanda','rw')
trans.translated_data()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id,utterance,slot_labels,intent,utterance_Kinyarwanda,slot_labels_Kinyarwanda,intent_Kinyarwanda
0,1,how much does the american airlines flight 71 ...,O O O O B-airline_name I-airline_name O B-flig...,airfare,angahe indege yindege yabanyamerika iguruka 71...,OOOO B-indege_nzina I-indege_nzina O B-indege_...,indege
1,2,could you tell me the cheapest fare from atlan...,O O O O O B-cost_relative O O B-fromloc.city_n...,airfare,ushobora kumbwira ibiciro bihendutse kuva atla...,OOOOO B-igiciro_guhuza OO B-kuva.city_name O B...,indege
2,3,what is the cost of flight dl 106 from philade...,O O O O O O B-airline_code B-flight_number O B...,airfare,nigiciro cyindege dl 106 kuva philadelphia kug...,OOOOOO B-indege_code B-indege_umubare O B-kuva...,indege
3,4,what is the cheapest round trip fare between b...,O O O B-cost_relative B-round_trip I-round_tri...,airfare,niyihe ngendo zihenze zihenze hagati ya boston...,OOO B.,indege
4,5,what is the fare from boston to oakland on uni...,O O O O O B-fromloc.city_name O B-toloc.city_n...,airfare,nigiciro ki kuva boston kugera oakland kumasos...,OOOOO B-kuva.city_name O B-toloc.city_name O B...,indege


In [16]:
# lug,'lg','Luganda'
trans = Translations(train_data[5:10],'Luganda','LG')
trans.translated_data()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id,utterance,slot_labels,intent,utterance_Luganda,slot_labels_Luganda,intent_Luganda
5,6,show me round trip fares from denver to philad...,O O B-round_trip I-round_trip O O B-fromloc.ci...,airfare,ndaga ebisale by&#39;okudda n&#39;okudda okuva...,OO B-olugendo_okudda I-olugendo_okudda OO B-ok...,ebisale by’ennyonyi
6,7,show me round trip fares from san jose to salt...,O O B-round_trip I-round_trip O O B-fromloc.ci...,airfare,ndaga ebisale by&#39;okudda n&#39;okudda okuva...,OO B-olugendo_okudda I-olugendo_okudda OO B-ok...,ebisale by’ennyonyi
7,8,show me us air fares for next sunday from miam...,O O B-airline_name I-airline_name O O B-depart...,airfare,tulage ebisale by&#39;ennyonyi ku sunday ejja ...,OO B-erinnya_ly&#39;ennyonyi I-erinnya_ly&#39;...,ebisale by’ennyonyi
8,9,list a round trip fare from kansas city to min...,O O B-round_trip I-round_trip O O B-fromloc.ci...,airfare,list a round trip fare okuva mu kansas city ok...,OO B-olugendo_okudda I-olugendo_okudda OO B-ok...,ebisale by’ennyonyi
9,10,what is the coach fare between boston and atla...,O O O B-class_type O O B-fromloc.city_name O B...,airfare,ssente za coach wakati wa boston ne atlanta ku...,OOO B-ekika_ky&#39;ekika OO B-okuva.erinnya_ly...,ebisale by’ennyonyi


In [19]:
# twi
trans = Translations(train_data[10:15],'Twi','ak')
trans.translated_data()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id,utterance,slot_labels,intent,utterance_Twi,slot_labels_Twi,intent_Twi
10,11,i would like your rates between atlanta and bo...,O O O O O O B-fromloc.city_name O B-toloc.city...,airfare,mepɛ sɛ wo rates wɔ atlanta ne boston ntam wɔ ...,OOOOOO B-fromloc.kurow_din O B-toloc.kurow_din...,wimhyɛn ho ka
11,12,please show me airfare of flight us 345 from b...,O O O O O O B-airline_code B-flight_number O B...,airfare,mesrɛ wo kyerɛ me wimhyɛn ka a ɛfa wimhyɛn us ...,OOOOOO B-wimhyɛn_kodo B-wimhyɛn_nɔma O B-filoc...,wimhyɛn ho ka
12,13,find the cheapest one way fare from boston to ...,O O B-cost_relative B-round_trip I-round_trip ...,airfare,hwehwɛ ɔkwan biako so ka a ne bo yɛ mmerɛw sen...,OO B-ɛka_abusuabɔ B-akwantuo_akɔhwɛ I-akwantuo...,wimhyɛn ho ka
13,14,please give me round trip fares from pittsburg...,O O O B-round_trip I-round_trip O O B-fromloc....,airfare,mesrɛ wo ma me akwantuo ho ka a ɛfiri pittsbur...,OOO B-akwantuo_akwantuo I-akwantuo_akwantuo_ak...,wimhyɛn ho ka
14,15,what is the price of american airlines flight ...,O O O O O B-airline_name I-airline_name O B-fl...,airfare,dɛn ne american airlines wimhyɛn 19 a efi new ...,OOOOO B-wimhyɛn_din I-wimhyɛn_din O B-wimhyɛn_...,wimhyɛn ho ka


In [20]:
# Swahili
trans = Translations(train_data[15:20],'Swahili','sw')
trans.translated_data()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id,utterance,slot_labels,intent,utterance_Swahili,slot_labels_Swahili,intent_Swahili
15,16,show me the coach fares from baltimore to dallas,O O O B-class_type O O B-fromloc.city_name O B...,airfare,nionyeshe nauli za kocha kutoka baltimore hadi...,OOO B-class_aina OO B-fromloc.city_name O B-to...,nauli ya ndege
16,17,could you please give me a one way fare from p...,O O O O O O B-round_trip I-round_trip O O B-fr...,airfare,tafadhali unaweza kunipa nauli ya njia moja ku...,OOOOOO B-safari_ya_kwenda na kurudi OO B-froml...,nauli ya ndege
17,18,give me fares from atlanta to baltimore,O O O O B-fromloc.city_name O B-toloc.city_name,airfare,nipe nauli kutoka atlanta hadi baltimore,OOOO B-fromloc.city_name O B-toloc.jina_la_jiji,nauli ya ndege
18,19,what is the lowest price fare from atlanta to ...,O O O B-cost_relative I-cost_relative O O B-fr...,airfare,ni nauli gani ya bei ya chini kutoka atlanta h...,OOO B-cost_relative I-cost_relative OO B-froml...,nauli ya ndege
19,20,what is the airfare between denver and pittsburgh,O O O O O B-fromloc.city_name O B-toloc.city_name,airfare,ni nini nauli ya ndege kati ya denver na pitts...,OOOOO B-fromloc.city_name O B-toloc.jina_la_jiji,nauli ya ndege
